# <center> Pipelines, Serialization </center> 

<p style="align: center;"><img align=center src="./images/Industrial-Pipeline-Cleaning.jpg"  width=1000></p>

.

.

<p style="align: center;"><img align=center src="https://hazelcast.com/wp-content/uploads/2019/11/24_DataPipleline-1.png"  width=1000></p>

In [221]:
import pandas as pd
import numpy as np

# Пара библиотек для визуализации
import matplotlib.pyplot as plt
%matplotlib inline


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

# Импорт Данных

In [222]:
file_path = 'https://raw.githubusercontent.com/a-milenkin/Datasetes_for_Piplines/main/responses.csv'
data = pd.read_csv(file_path)

In [223]:
data.head()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow


<div class="alert alert-info">

<b>Про датасет </b>


    
Приведеные результаты опроса молодежи до 21-го года по тому, какие вещи делют их счастливыми по 5-й шкале
    
</div>


<div class="alert alert-info">

<b>Про Задачу </b>

Попробуем классифицировать людей по оценкам уровня счастья
    
</div>


In [224]:
interesting_features = ["Happiness in life", "Age", "Height", "Gender", "Weight", 
              "Left - right handed", "Village - town", "Getting up", "God",
              "Health", "Borrowed stuff", "Self-criticism", "Elections", 
              "Smoking", "Alcohol" ,"Number of friends", "Spending on healthy eating",
              "Music", "Movies", "Friends versus money", "Changing the past"]

target = ["Happiness in life"]

In [225]:
# data["Happiness in life"].hist();

In [226]:
data = data[interesting_features]

In [227]:
# Сбалансируем немного классы

def happiness_score(x):
    if x < 4:
        return -1
    elif x == 4.0:
        return 0
    else:
        return 1
    
data["Happiness in life"] = data["Happiness in life"].apply(happiness_score)

# Как пишут код обычно? - The painful way

# Какой-нибудь EDA (исследование данных)

# Есть ли в данных выбросы? Есть почти всегда!

In [228]:
# data.isnull().sum()

In [229]:
# data.dropna(inplace=True)

In [230]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


data_train, data_test = train_test_split(data.copy(), test_size=0.2, stratify=data[target])

data_train.dropna(inplace=True)
data_test.dropna(inplace=True)

print(data_train.shape, data_test.shape)

(738, 21) (187, 21)


In [231]:
data.head()

,Happiness in life,Age,Height,Gender,Weight,Left - right handed,Village - town,Getting up,God,Health,...,Self-criticism,Elections,Smoking,Alcohol,Number of friends,Spending on healthy eating,Music,Movies,Friends versus money,Changing the past
0,0,20.0,163.0,female,48.0,right handed,village,2.0,1.0,1.0,...,1.0,4.0,never smoked,drink a lot,3,3.0,5.0,5.0,3.0,1.0
1,0,19.0,163.0,female,58.0,right handed,city,5.0,1.0,4.0,...,4.0,5.0,never smoked,drink a lot,3,2.0,4.0,5.0,4.0,4.0
2,0,20.0,176.0,female,67.0,right handed,city,4.0,5.0,2.0,...,4.0,5.0,tried smoking,drink a lot,3,2.0,5.0,5.0,5.0,5.0
3,-1,22.0,172.0,female,59.0,right handed,city,1.0,4.0,1.0,...,5.0,5.0,former smoker,drink a lot,1,1.0,5.0,5.0,2.0,5.0
4,-1,20.0,170.0,female,59.0,right handed,village,4.0,5.0,3.0,...,5.0,5.0,tried smoking,social drinker,3,4.0,5.0,5.0,3.0,4.0


In [232]:
data['Alcohol'].value_counts()

social drinker    659
drink a lot       222
never             124
Name: Alcohol, dtype: int64

# Какие признаки в данных?

In [233]:
numerical_features = ['Age', 'Height', 'Weight']  # Численные переменные

# Бинарные признаки
to_dummies = [
    'Gender', 'Left - right handed', 'Village - town', 'Smoking',  "Alcohol" ,
]

# Категориальные признаки
categorical_features = [
    col for col in data.columns if col not in numerical_features + target + to_dummies
]

all_features = categorical_features + numerical_features
len(all_features)

15

In [234]:
# data[numerical_features]

In [235]:
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

In [236]:
scaler = StandardScaler() # Нормировка для численных признаков

In [237]:
data_train_scaled = scaler.fit_transform(data_train[numerical_features])
data_train_scaled[:10]

array([[-0.48530076, -1.00013871, -1.18913612],
       [ 2.34594006,  0.91055859,  0.96074741],
       [-0.83920587,  1.38823292,  1.67737526],
       [-1.90092117,  0.71948886, -0.25751992],
       [-1.54701607,  0.623954  , -0.8308222 ],
       [ 0.22250944, -1.00013871, -0.8308222 ],
       [-0.48530076, -0.33139465,  0.60243349],
       [ 2.69984516,  0.81502373,  1.53404969],
       [ 0.22250944, -0.04479006, -0.97414777],
       [-0.13139566,  0.43288427, -0.11419436]])

In [238]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')  # кодирование для категориальных признаков

data_train_ohe = ohe.fit_transform(data_train[to_dummies])
data_train_ohe[:10]

array([[1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.]])

In [239]:
data_train_tramsformed = pd.concat([pd.DataFrame(data_train_scaled, columns=numerical_features),
                                    pd.DataFrame(data_train_ohe, columns=ohe.get_feature_names()),
                                   # pd.DataFrame(data_train[categorical_features].values)
                                   ], axis=1)
data_train_tramsformed.head()

,Age,Height,Weight,x0_female,x0_male,x1_left handed,x1_right handed,x2_city,x2_village,x3_current smoker,x3_former smoker,x3_never smoked,x3_tried smoking,x4_drink a lot,x4_never,x4_social drinker
0,-0.485301,-1.000139,-1.189136,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.345940,0.910559,0.960747,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,-0.839206,1.388233,1.677375,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,-1.900921,0.719489,-0.257520,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,-1.547016,0.623954,-0.830822,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


# ML часть

In [240]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer

In [241]:
model = LogisticRegression()

model.fit(data_train_tramsformed, data_train[target])

LogisticRegression()

In [242]:
preds = model.predict(data_train_tramsformed)


print(classification_report(data_train[target], preds))

              precision    recall  f1-score   support

          -1       1.00      0.01      0.02       258
           0       0.51      1.00      0.67       373
           1       0.00      0.00      0.00       107

    accuracy                           0.51       738
   macro avg       0.50      0.34      0.23       738
weighted avg       0.61      0.51      0.35       738



# А как же тест?! Валидация

In [243]:
data_test_scaled = scaler.transform(data_test[numerical_features])
data_test_scaled[:10]

array([[ 0.22250944, -0.04479006,  0.45910792],
       [ 0.57641454, -1.38227817, -0.97414777],
       [ 2.69984516,  0.24181454, -0.47250828],
       [-0.48530076, -0.52246438, -0.68749663],
       [ 0.22250944,  0.623954  ,  0.24411957],
       [ 0.57641454, -0.04479006,  0.96074741],
       [ 0.22250944, -1.19120844, -1.33246169],
       [ 0.22250944, -0.90460384, -1.04581055],
       [-0.48530076,  0.14627967, -0.47250828],
       [-1.19311097,  1.10162832,  1.10407298]])

In [244]:
data_test_ohe = ohe.transform(data_test[to_dummies])
data_test_ohe[:10]

array([[0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
       [1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.]])

In [245]:
data_test_tramsformed = pd.concat([pd.DataFrame(data_test_scaled, columns=numerical_features),
                                    pd.DataFrame(data_test_ohe, columns=ohe.get_feature_names()),
                                    #pd.DataFrame(data_test[categorical_features].values)
                                  ], axis=1)
data_test_tramsformed.head()

,Age,Height,Weight,x0_female,x0_male,x1_left handed,x1_right handed,x2_city,x2_village,x3_current smoker,x3_former smoker,x3_never smoked,x3_tried smoking,x4_drink a lot,x4_never,x4_social drinker
0,0.222509,-0.044790,0.459108,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.576415,-1.382278,-0.974148,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,2.699845,0.241815,-0.472508,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.485301,-0.522464,-0.687497,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.222509,0.623954,0.244120,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [246]:
preds = model.predict(data_test_tramsformed)

In [247]:
print(classification_report(data_test[target], preds))

              precision    recall  f1-score   support

          -1       0.50      0.02      0.03        66
           0       0.50      0.98      0.66        94
           1       0.00      0.00      0.00        27

    accuracy                           0.50       187
   macro avg       0.33      0.33      0.23       187
weighted avg       0.43      0.50      0.34       187



## Что не так с этим способом?

<div class="alert alert-warning">


 - Дублируем код ли кода

 - Сложно маштабировать

 - Сложно поддерживать свой код
    
 - Много времени(

Как сохранить всю эту махину? 

# Как пишут код true-датасаентисты - The pipeline way 

<div>
<img src="https://i.pinimg.com/564x/41/d2/4b/41d24b25710d7746b22a7eb34316952c.jpg" width="500"/>
</div>

<div class="alert alert-info">

Pipeline позволяет обернуть сразу несколько этапов обработки данных в один объект

- **FunctionTransformer**: преобразует функцию в трансформер
- **FeatureUnion**: объединяет результаты нескольких трансформеров в один датасет
- **ColumnTransformer**: выбирает определенные колонки
- **make_pipeline**: удобная обертка для создания пайплайна, можно использовать вместо Pipeline 

In [248]:
# Гости этого вечера. Нужные нам фрагменты для сборки пайплайна

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif

In [249]:
#   Для категориальных - применяем SimpleImputer, а затем StandardScaler
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(score_func=f_classif, k="all")),
])

#   Для категориальных - применяем SimpleImputer, а затем OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features),
])

# Создание конвейера препроцессора, который сначала преобразует данные
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

classifier_pipline = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", LogisticRegression())])

classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler()),
                                                                                   ('fs',
                                                                                    SelectKBest(k='all'))]),
                                                                   ['Age',
                                                                    'Height',
                                                                    'Weight']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['Getting '
                                                                    'up',
                                                                    'God',
                                                                    'Health',
                                                                    'Borrowed '
                                                                    'stuff',
                                                                    'Self-criticism',
                                                                    'Elections',
                                                                    'Number of '
                                                                    'friends',
                                                                    'Spending '
                                                                    'on '
                                                                    'healthy '
                                                                    'eating',
                                                                    'Music',
                                                                    'Movies',
                                                                    'Friends '
                                                                    'versus '
                                                                    'money',
                                                                    'Changing '
                                                                    'the '
                                                                    'past'])]))])),
                ('classifier', LogisticRegression())])

# Добавим магии - отрисовка!

In [250]:
import sklearn
sklearn.set_config(display='diagram') #Подключим визуал

In [251]:
classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler()),
                                                                                   ('fs',
                                                                                    SelectKBest(k='all'))]),
                                                                   ['Age',
                                                                    'Height',
                                                                    'Weight']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['Getting '
                                                                    'up',
                                                                    'God',
                                                                    'Health',
                                                                    'Borrowed '
                                                                    'stuff',
                                                                    'Self-criticism',
                                                                    'Elections',
                                                                    'Number of '
                                                                    'friends',
                                                                    'Spending '
                                                                    'on '
                                                                    'healthy '
                                                                    'eating',
                                                                    'Music',
                                                                    'Movies',
                                                                    'Friends '
                                                                    'versus '
                                                                    'money',
                                                                    'Changing '
                                                                    'the '
                                                                    'past'])]))])),
                ('classifier', LogisticRegression())])

In [252]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data[all_features + target], test_size=0.2, stratify=data[target])
print(data_train.shape, data_test.shape)

(808, 16) (202, 16)


In [253]:
# # data_train.columns
# all_features

In [254]:
# data_train[all_features]

In [255]:

classifier_pipline.fit(data_train[all_features], data_train[target])

preds = classifier_pipline.predict(data_test[all_features])

print(classification_report(data_test[target], preds))

              precision    recall  f1-score   support

          -1       0.51      0.51      0.51        71
           0       0.62      0.72      0.66       102
           1       0.64      0.31      0.42        29

    accuracy                           0.58       202
   macro avg       0.59      0.51      0.53       202
weighted avg       0.59      0.58      0.57       202



# А как подбирать параметры у этой всей махины??

In [256]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score, average="macro")

In [257]:
param_grid = {
    'classifier__C': np.logspace(-5, 2, 200), # np.linspace(-10, 10)
    'classifier__penalty': ['l1', 'l2'],
    'classifier__class_weight': ['balanced', None],
    'preprocessor__data_transformer__numerical__imputer__strategy':['median', 'mean']
}


search = GridSearchCV(classifier_pipline, param_grid, n_jobs=-1, cv=3) #, scoring=f1)

search.fit(data_train.drop(target, axis=1), data_train[target])

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('data_transformer',
                                                         ColumnTransformer(transformers=[('numerical',
                                                                                          Pipeline(steps=[('imputer',
                                                                                                           SimpleImputer()),
                                                                                                          ('scaler',
                                                                                                           StandardScaler()),
                                                                                                          ('fs',
                                                                                                           SelectKBest(k='all'))]),
                                                                                          ['Age',
                                                                                           'Height',
                                                                                           'Weight']),
                                                                                         ('categorical',
                                                                                          Pipeline(steps=[('imputer',
                                                                                                           SimpleImputer()),
                                                                                                          ('onehot',
                                                                                                           OneHotEncod...
       2.96730241e+01, 3.21764175e+01, 3.48910121e+01, 3.78346262e+01,
       4.10265811e+01, 4.44878283e+01, 4.82410870e+01, 5.23109931e+01,
       5.67242607e+01, 6.15098579e+01, 6.66991966e+01, 7.23263390e+01,
       7.84282206e+01, 8.50448934e+01, 9.22197882e+01, 1.00000000e+02]),
                         'classifier__class_weight': ['balanced', None],
                         'classifier__penalty': ['l1', 'l2'],
                         'preprocessor__data_transformer__numerical__imputer__strategy': ['median',
                                                                                          'mean']})

<div class="alert alert-info">

Нам даже не нужно теперь делить на `train` и `test`, ведь есть валидация 

In [258]:
search.best_params_

{'classifier__C': 0.06294988990221888,
 'classifier__class_weight': None,
 'classifier__penalty': 'l2',
 'preprocessor__data_transformer__numerical__imputer__strategy': 'median'}

In [259]:
search.best_score_

0.5544724402221304

# Еще раз, но по шагам!

In [260]:
classifier_pipline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('classifier', LogisticRegression())])

In [261]:
# Создание конвейера препроцессора, который сначала преобразует данные.
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])


# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numerical_transformer, numerical_features),
        ("categorical", categorical_transformer, categorical_features),
    ]
)

In [262]:
#   Для категориальных - применяем SimpleImputer, а затем StandardScaler

numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler()),
        ("fs", SelectKBest(score_func=f_classif, k="all")),
    ]
)

In [263]:
#   Для категориальных - применяем SimpleImputer, а затем OneHotEncoder
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)


classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler()),
                                                                                   ('fs',
                                                                                    SelectKBest(k='all'))]),
                                                                   ['Age',
                                                                    'Height',
                                                                    'Weight']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['Getting '
                                                                    'up',
                                                                    'God',
                                                                    'Health',
                                                                    'Borrowed '
                                                                    'stuff',
                                                                    'Self-criticism',
                                                                    'Elections',
                                                                    'Number of '
                                                                    'friends',
                                                                    'Spending '
                                                                    'on '
                                                                    'healthy '
                                                                    'eating',
                                                                    'Music',
                                                                    'Movies',
                                                                    'Friends '
                                                                    'versus '
                                                                    'money',
                                                                    'Changing '
                                                                    'the '
                                                                    'past'])]))])),
                ('classifier', LogisticRegression())])

# А если вдруг одной модели мало?

In [264]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost

clf1 = LogisticRegression(multi_class="multinomial", random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

blending_classifier = VotingClassifier(
    estimators=[("log_regrassion", clf1), 
                ("random_forest", clf2), 
                ("gnb", clf3)] 
)
blending_classifier

VotingClassifier(estimators=[('log_regrassion',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('random_forest',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=1)),
                             ('gnb', GaussianNB())])

In [265]:
classifier_pipline = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("classifier", blending_classifier)]
)

classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler()),
                                                                                   ('fs',
                                                                                    SelectKBest(k='all'))]),
                                                                   ['Age',
                                                                    'Height',
                                                                    'Weight']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore'))...
                                                                    'Elections',
                                                                    'Number of '
                                                                    'friends',
                                                                    'Spending '
                                                                    'on '
                                                                    'healthy '
                                                                    'eating',
                                                                    'Music',
                                                                    'Movies',
                                                                    'Friends '
                                                                    'versus '
                                                                    'money',
                                                                    'Changing '
                                                                    'the '
                                                                    'past'])]))])),
                ('classifier',
                 VotingClassifier(estimators=[('log_regrassion',
                                               LogisticRegression(multi_class='multinomial',
                                                                  random_state=1)),
                                              ('random_forest',
                                               RandomForestClassifier(n_estimators=50,
                                                                      random_state=1)),
                                              ('gnb', GaussianNB())]))])

# А что если хотим использовать стекинг?

In [266]:
# Your code here.
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier


In [267]:
# получим список эстиматоров
estimators = [
    ("Random_Forest", RandomForestClassifier()),
    ("Xgboost", xgboost.XGBClassifier()),
    ("SVM", LinearSVC()),
    
]

# построим классификатор используя стекинг и наш список классификаторов
stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
    n_jobs=-1,
    verbose=True,
)

In [268]:
stacking_classifier

StackingClassifier(estimators=[('Random_Forest', RandomForestClassifier()),
                               ('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=None,
                                              max_depth=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, reg_alpha=None,
                                              reg_lambda=None,
                                              scale_pos_weight=None,
                                              subsample=None, tree_method=None,
                                              validate_parameters=None,
                                              verbosity=None)),
                               ('SVM', LinearSVC())],
                   final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
                   n_jobs=-1, verbose=True)

In [269]:
# получим список эстиматоров
estimators = [
    ("Random_Forest", RandomForestClassifier()),
    ("SVM", LinearSVC()),
    ("Xgboost", xgboost.XGBClassifier()),
]

# построим классификатор используя стекинг и наш список классификаторов
stacking_classifier = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(n_jobs=-1), n_jobs=-1
)

classifier_pipline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", stacking_classifier)]
)

classifier_pipline

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler()),
                                                                                   ('fs',
                                                                                    SelectKBest(k='all'))]),
                                                                   ['Age',
                                                                    'Height',
                                                                    'Weight']),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('onehot',
                                                                                    OneHotEncoder(handle_unknown='ignore'))...
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None,
                                                               scale_pos_weight=None,
                                                               subsample=None,
                                                               tree_method=None,
                                                               validate_parameters=None,
                                                               verbosity=None))],
                                    final_estimator=LogisticRegression(n_jobs=-1),
                                    n_jobs=-1))])

# Чуть более хитрый стекинг/блендинг моделей

In [270]:
from sklearn.decomposition import PCA

# получим список эстиматоров
estimators = [
    ("SVM", make_pipeline(preprocessor, PCA(), LinearSVC())),
    ("Random_Forest", make_pipeline(preprocessor, RandomForestClassifier())),
    ("Xgboost", make_pipeline(preprocessor, xgboost.XGBClassifier())),
]

# построим классификатор используя стекинг и наш список классификаторов
stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
    n_jobs=-1,
    verbose=True,
)

stacking_classifier

StackingClassifier(estimators=[('SVM',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('data_transformer',
                                                                  ColumnTransformer(transformers=[('numerical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   ('scaler',
                                                                                                                    StandardScaler()),
                                                                                                                   ('fs',
                                                                                                                    SelectKBest(k='all'))]),
                                                                                                   ['Age',
                                                                                                    'Height',
                                                                                                    'Weight']),
                                                                                                  ('categorical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   ('onehot',
                                                                                                                    OneHo...
                                                               max_depth=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None,
                                                               scale_pos_weight=None,
                                                               subsample=None,
                                                               tree_method=None,
                                                               validate_parameters=None,
                                                               verbosity=None))]))],
                   final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
                   n_jobs=-1, verbose=True)

# А сколько всего кода было написано?

In [271]:
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("feature_selector", SelectKBest(score_func=f_classif, k="all"))])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

data_transformer = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

estimators = [
    ("SVM", make_pipeline(preprocessor, LinearSVC())),
    ("Random_Forest", make_pipeline(preprocessor, RandomForestClassifier(n_jobs=-1))),
    ("xgboost", make_pipeline(preprocessor, xgboost.XGBClassifier(n_jobs=-1)))]

stacking_classifier = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
    n_jobs=-1,
    verbose=True,
)

In [272]:
stacking_classifier

StackingClassifier(estimators=[('SVM',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('data_transformer',
                                                                  ColumnTransformer(transformers=[('numerical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   ('scaler',
                                                                                                                    StandardScaler()),
                                                                                                                   ('feature_selector',
                                                                                                                    SelectKBest(k='all'))]),
                                                                                                   ['Age',
                                                                                                    'Height',
                                                                                                    'Weight']),
                                                                                                  ('categorical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   (...
                                                               max_depth=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=-1,
                                                               num_parallel_tree=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None,
                                                               scale_pos_weight=None,
                                                               subsample=None,
                                                               tree_method=None,
                                                               validate_parameters=None,
                                                               verbosity=None))]))],
                   final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
                   n_jobs=-1, verbose=True)

# Обучении махины - Будет ли легко?

In [273]:
stacking_classifier.fit(data[interesting_features], data[target])


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


StackingClassifier(estimators=[('SVM',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('data_transformer',
                                                                  ColumnTransformer(transformers=[('numerical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   ('scaler',
                                                                                                                    StandardScaler()),
                                                                                                                   ('feature_selector',
                                                                                                                    SelectKBest(k='all'))]),
                                                                                                   ['Age',
                                                                                                    'Height',
                                                                                                    'Weight']),
                                                                                                  ('categorical',
                                                                                                   Pipeline(steps=[('imputer',
                                                                                                                    SimpleImputer()),
                                                                                                                   (...
                                                               max_depth=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=-1,
                                                               num_parallel_tree=None,
                                                               random_state=None,
                                                               reg_alpha=None,
                                                               reg_lambda=None,
                                                               scale_pos_weight=None,
                                                               subsample=None,
                                                               tree_method=None,
                                                               validate_parameters=None,
                                                               verbosity=None))]))],
                   final_estimator=LogisticRegression(n_jobs=-1, verbose=True),
                   n_jobs=-1, verbose=True)

In [274]:
preds = stacking_classifier.predict(data[interesting_features])

print(classification_report(data[target], preds))

              precision    recall  f1-score   support

          -1       0.98      0.95      0.97       355
           0       0.96      1.00      0.98       509
           1       1.00      0.93      0.96       146

    accuracy                           0.97      1010
   macro avg       0.98      0.96      0.97      1010
weighted avg       0.97      0.97      0.97      1010



# Гибкость паплайна

In [275]:
# stacking_classifier.estimators[0][1][0].steps[0][1].transformers[0][1].steps[2][1].k

# Как сохранить всю эту махину?

In [276]:
# Для сериализации моделей
import pickle

In [277]:
with open("owesome_pipeline.pkl", "wb") as f:
    pickle.dump(stacking_classifier, f)

# Импорт модели

In [278]:
with open("owesome_pipeline.pkl", 'rb') as f:
    pipeline_from_saved = pickle.load(f)

In [279]:
pipeline_from_saved.fit(data_train, data_train[target])
preds = pipeline_from_saved.predict(data_test)
print(classification_report(data_test[target], preds))

              precision    recall  f1-score   support

          -1       0.55      0.41      0.47        71
           0       0.59      0.81      0.68       102
           1       1.00      0.28      0.43        29

    accuracy                           0.59       202
   macro avg       0.71      0.50      0.53       202
weighted avg       0.63      0.59      0.57       202



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Что дальше?

<div class="alert alert-success">
    
* Признаки бывают разные
* Предобработка бывает сложнее
* Генерация новых признаков

<p style="align: center;"><img align=center src="./images/Industrial-Pipeline-Cleaning.jpg"  width=1000></p>

In [280]:
import pandas as pd
import numpy as np

# Пара библиотек для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import sklearn
sklearn.set_config(display='diagram') # Подключим визуал

import warnings
warnings.filterwarnings("ignore")

# Кастомизированные опции

In [281]:
# Гости этого вечера. Нужные нам фрагменты для сборки пайплайна, они пригодятся нам позже

# from sklearn.pipeline import (
#     _fit_transform_one,
#     _transform_one,
# )

# И уже знакомые нам части 
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

In [282]:
file_path = 'https://raw.githubusercontent.com/a-milenkin/Datasetes_for_Piplines/main/responses.csv'
data = pd.read_csv(file_path)
data.head(3)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats


<div class="alert alert-info">

<b>Про датасет </b>


    
Приведеные результаты опроса молодежи до 21-го года по тому, какие вещи делют их счастливыми по 5-й шкале
    
</div>


<div class="alert alert-info">

<b>Про Задачу </b>

Попробуем классифицировать людей по оценкам уровня счастья
    
</div>


In [283]:
# Сбалансируем немного классы

def happiness_score(x):
    if x < 4:
        return -1
    elif x == 4.0:
        return 0
    else:
        return 1
    
data["Happiness in life"] = data["Happiness in life"].apply(happiness_score)

# Классификаций признаков на типы - df.select_dtypes

In [284]:
data.select_dtypes(exclude = ['int', 'float']).info()  # include

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Smoking                 1002 non-null   object
 1   Alcohol                 1005 non-null   object
 2   Punctuality             1008 non-null   object
 3   Lying                   1008 non-null   object
 4   Internet usage          1010 non-null   object
 5   Gender                  1004 non-null   object
 6   Left - right handed     1007 non-null   object
 7   Education               1009 non-null   object
 8   Only child              1008 non-null   object
 9   Village - town          1006 non-null   object
 10  House - block of flats  1006 non-null   object
dtypes: object(11)
memory usage: 86.9+ KB


In [285]:
numerical_features = data.select_dtypes(include=["int", "float"]).columns.to_list()

data[numerical_features].head()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,4.0,5.0,3.0,3.0,1,3.0,20.0,163.0,48.0,1.0
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,4.0,1.0,4.0,2.0,5,2.0,19.0,163.0,58.0,2.0
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,4.0,1.0,4.0,3.0,4,2.0,20.0,176.0,67.0,2.0
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,4.0,3.0,3.0,4.0,4,1.0,22.0,172.0,59.0,1.0
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,3.0,4.0,3.0,3.0,2,4.0,20.0,170.0,59.0,1.0


In [286]:
data.select_dtypes(exclude=["int", "float"]).head()

,Smoking,Alcohol,Punctuality,Lying,Internet usage,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,never smoked,drink a lot,i am always on time,never,few hours a day,female,right handed,college/bachelor degree,no,village,block of flats
1,never smoked,drink a lot,i am often early,sometimes,few hours a day,female,right handed,college/bachelor degree,no,city,block of flats
2,tried smoking,drink a lot,i am often running late,sometimes,few hours a day,female,right handed,secondary school,no,city,block of flats
3,former smoker,drink a lot,i am often early,only to avoid hurting someone,most of the day,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,tried smoking,social drinker,i am always on time,everytime it suits me,few hours a day,female,right handed,secondary school,no,village,house/bungalow


In [287]:
binary_features = []
category_features  = []

for col in data.select_dtypes(exclude = ['int', 'float']).columns: 
    if data.dropna()[col].nunique() == 2:
        binary_features.append(col)
    else:
        category_features.append(col)

In [288]:
data[category_features].head(3)

,Smoking,Alcohol,Punctuality,Lying,Internet usage,Education
0,never smoked,drink a lot,i am always on time,never,few hours a day,college/bachelor degree
1,never smoked,drink a lot,i am often early,sometimes,few hours a day,college/bachelor degree
2,tried smoking,drink a lot,i am often running late,sometimes,few hours a day,secondary school


In [289]:
data[binary_features].head(3)

,Gender,Left - right handed,Only child,Village - town,House - block of flats
0,female,right handed,no,village,block of flats
1,female,right handed,no,city,block of flats
2,female,right handed,no,city,block of flats


# Все,теперь у нас три типа признаков

In [290]:
interesting_features = numerical_features + category_features + binary_features

target = ["Happiness in life"]

data = data[interesting_features]

In [291]:
data.head(3)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Alcohol,Punctuality,Lying,Internet usage,Education,Gender,Left - right handed,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,drink a lot,i am always on time,never,few hours a day,college/bachelor degree,female,right handed,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,drink a lot,i am often early,sometimes,few hours a day,college/bachelor degree,female,right handed,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,drink a lot,i am often running late,sometimes,few hours a day,secondary school,female,right handed,no,city,block of flats


# Numerical features

In [292]:
data[numerical_features].head(2)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,4.0,5.0,3.0,3.0,1,3.0,20.0,163.0,48.0,1.0
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,4.0,1.0,4.0,2.0,5,2.0,19.0,163.0,58.0,2.0


In [293]:
from sklearn.preprocessing import FunctionTransformer

In [294]:
# Тот же способ через функцию
def numerical_select(data):

    data['random_values'] = np.random.normal(size = len(data))
    
    return data[numerical_features + ['random_values']]

# Превращаем функцию в трансформер
numerical_selector = FunctionTransformer(numerical_select, validate=False)

numerical_selector.fit_transform(data).head(3)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings,random_values
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,5.0,3.0,3.0,1,3.0,20.0,163.0,48.0,1.0,0.187485
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,1.0,4.0,2.0,5,2.0,19.0,163.0,58.0,2.0,-0.974446
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,1.0,4.0,3.0,4,2.0,20.0,176.0,67.0,2.0,-0.463639


In [295]:
# Тот же способ через лямбда-функцию
# numerical_selector = FunctionTransformer(lambda data: data[numerical_features], validate=False)

In [296]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()

In [297]:
numerical_preprocessor = Pipeline([("numerical_selector", numerical_selector),
                                   ("scaling", scaler)])

In [298]:
numerical_preprocessor.fit_transform(data) 
# Куда-то пропали столцы, надо исправлять!!!

array([[ 0.40397078, -0.39396081, -0.95156664, ..., -1.33055906,
        -0.29403239,  0.17880542],
       [-1.10269062,  0.80577482, -0.95156664, ..., -0.60762757,
         0.69328707, -0.13123147],
       [ 0.40397078,  2.00551046, -0.95156664, ...,  0.04301077,
         0.69328707,  0.52318059],
       ...,
       [-1.10269062, -0.39396081, -1.80627739, ...,  0.62135597,
        -1.28135184, -0.1533262 ],
       [ 0.40397078, -0.39396081, -0.09685589, ..., -0.60762757,
        -0.29403239, -0.24134634],
       [ 0.40397078,  2.00551046,  0.75785486, ...,  0.40447652,
        -0.29403239,  0.51458814]])

In [299]:
# Pipeline в отличии от make_pipline дает возможность обращася к объекту пайплайна по названию
numerical_preprocessor.named_steps['scaling'] #

StandardScaler()

In [300]:
class StandardScalerDf(StandardScaler):
    """
    DataFrame Wrapper around StandardScaler
    http://flennerhag.com/2017-01-08-Recursive-Override/
    """
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super(StandardScalerDf, self).__init__(copy=copy,
                                               with_mean=with_mean,
                                               with_std=with_std)
        self.feature_names = None

    def transform(self, X, y=None):
        self.feature_names = X.columns
        
        z = super(StandardScalerDf, self).transform(X.values)
        
        return pd.DataFrame(z, index=X.index, columns=self.feature_names)
    
    def get_feature_names(self):
        return self.feature_names

In [301]:
scaler_df = StandardScalerDf()

In [302]:
numerical_preprocessor = Pipeline([
    ("numerical_selector", numerical_selector),
    ("scaling", scaler_df)
])

In [303]:
numerical_preprocessor.fit_transform(data).head(3) # Сохраняются названия столцов. Лушче, но можно сделать меньше кода

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings,random_values
0,0.403971,-0.393961,-0.951567,-1.131952,-0.114711,-0.763717,-1.398094,1.316567,1.04541,-0.992099,...,1.493027,-0.169637,-0.088196,-1.456239,-0.510004,-0.153390,-1.049374,-1.330559,-0.294032,-0.391297
1,-1.102691,0.805775,-0.951567,-1.131952,-1.044424,-1.562473,-0.604581,-0.406348,1.04541,1.193992,...,-1.570528,0.671861,-0.918230,1.658216,-1.424743,-0.507068,-1.049374,-0.607628,0.693287,-0.360333
2,0.403971,2.005510,-0.951567,-0.253487,0.815003,0.833797,1.775957,-0.406348,1.04541,0.465295,...,-1.570528,0.671861,-0.088196,0.879602,-1.424743,-0.153390,0.248104,0.043011,0.693287,1.227031


In [304]:
numerical_preprocessor

Pipeline(steps=[('numerical_selector',
                 FunctionTransformer(func=<function numerical_select at 0x000002279EF34AF0>)),
                ('scaling', StandardScalerDf())])

In [305]:
numerical_preprocessor.named_steps['numerical_selector']

FunctionTransformer(func=<function numerical_select at 0x000002279EF34AF0>)

# Бинарные признаки

In [306]:
# Способ через функцию 
def binary_select(data):
    
    data['random_binar'] = data['Gender'] == 'female'
    
    return data[binary_features + ['random_binar']] 

# Превращаем функцию в трансформер
binary_selector = FunctionTransformer(binary_select, validate=False)  


# Применяем к данным
binary_selector.fit_transform(data).head()

,Gender,Left - right handed,Only child,Village - town,House - block of flats,random_binar
0,female,right handed,no,village,block of flats,True
1,female,right handed,no,city,block of flats,True
2,female,right handed,no,city,block of flats,True
3,female,right handed,yes,city,house/bungalow,True
4,female,right handed,no,village,house/bungalow,True


In [307]:
# Тот же самый пособ, но через лябма-функцию / короткий способ
binary_selector = FunctionTransformer(lambda data: data[binary_features], validate=False)

In [308]:
# pd.get_dummies(data, columns=to_dummies)

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [309]:
binary_preprocessor = Pipeline([
    ("binary_selector", binary_selector),
    ("ohe", ohe)
])

In [310]:
class OneHotEncodingDf(OneHotEncoder):
    """DataFrame Wrapper around OneHotEncoding"""
    def __init__(self, drop=None, sparse=False, handle_unknown='ignore'):
        super(OneHotEncodingDf, self).__init__(drop=drop,
                                               sparse=sparse,
                                               handle_unknown=handle_unknown
                                              )

    def transform(self, X, y=None):
        z = super(OneHotEncodingDf, self).transform(X.values)
        return pd.DataFrame(z, index=X.index, columns=self.get_feature_names())

In [311]:
ohe_df = OneHotEncodingDf()

In [312]:
binary_preprocessor = Pipeline([
    ("binary_selector", binary_selector),
    ("ohe", ohe_df)
])

In [313]:
binary_preprocessor.fit_transform(data).head()

,x0_female,x0_male,x0_nan,x1_left handed,x1_right handed,x1_nan,x2_no,x2_yes,x2_nan,x3_city,x3_village,x3_nan,x4_block of flats,x4_house/bungalow,x4_nan
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


# Категориальные переменные

In [314]:
def categorical_select(data):
    return data[categorical_features]

categorical_selector = FunctionTransformer(categorical_select, validate=False)


# categorical_selector = FunctionTransformer(lambda data: data[categorical_features], validate=False)

In [315]:
categorical_preprocessor = Pipeline([
    ("categorical_selector", categorical_selector),
    ("ohe", OneHotEncodingDf())
])

# categorical_selector.fit_transform(data).head()

# Feature Union - собираем все в один датасет

In [316]:
feature_union = FeatureUnion([
    ("numerical_preprocessor", numerical_preprocessor),
    ("binary_preprocessor", binary_preprocessor),
    ("categorical_preprocessor", categorical_preprocessor)
])

In [317]:
feature_union

FeatureUnion(transformer_list=[('numerical_preprocessor',
                                Pipeline(steps=[('numerical_selector',
                                                 FunctionTransformer(func=<function numerical_select at 0x000002279EF34AF0>)),
                                                ('scaling',
                                                 StandardScalerDf())])),
                               ('binary_preprocessor',
                                Pipeline(steps=[('binary_selector',
                                                 FunctionTransformer(func=<function <lambda> at 0x00000227967BE310>)),
                                                ('ohe', OneHotEncodingDf())])),
                               ('categorical_preprocessor',
                                Pipeline(steps=[('categorical_selector',
                                                 FunctionTransformer(func=<function categorical_select at 0x0000022796FAEA60>)),
                                                ('ohe', OneHotEncodingDf())]))])

In [318]:
feature_union.fit_transform(data)

array([[ 0.40397078, -0.39396081, -0.95156664, ...,  0.        ,
         0.        ,  0.        ],
       [-1.10269062,  0.80577482, -0.95156664, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.40397078,  2.00551046, -0.95156664, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.10269062, -0.39396081, -1.80627739, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40397078, -0.39396081, -0.09685589, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40397078,  2.00551046,  0.75785486, ...,  0.        ,
         0.        ,  0.        ]])

In [319]:
feature_union.fit_transform(data).shape

(1010, 226)

In [320]:
# feature_union.named_steps['numerical_preprocessor']

In [321]:
from joblib import Parallel, delayed 
from scipy import sparse

# Пара новых гостей
from sklearn.pipeline import (
    _fit_transform_one,
    _transform_one,
)

# https://zablo.net/blog/post/pandas-dataframe-in-scikit-learn-feature-union/

class PandasFeatureUnion(FeatureUnion):
    def fit_transform(self, X, y=None, **fit_params):
        self._validate_transformers()
        result = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_transform_one)(
                transformer=trans,
                X=X,
                y=y,
                weight=weight,
                **fit_params)
            for name, trans, weight in self._iter())

        if not result:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        Xs, transformers = zip(*result)
        self._update_transformer_list(transformers)
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

    def merge_dataframes_by_column(self, Xs):
        return pd.concat(Xs, axis="columns", copy=False)

    def transform(self, X):
        Xs = Parallel(n_jobs=self.n_jobs)(
            delayed(_transform_one)(
                transformer=trans,
                X=X,
                y=None,
                weight=weight)
            for name, trans, weight in self._iter())
        if not Xs:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

In [322]:
feature_union = PandasFeatureUnion([
    ("numerical_preprocessor", numerical_preprocessor),
    ("binary_preprocessor", binary_preprocessor),
    ("categorical_preprocessor", categorical_preprocessor)
])

In [323]:
feature_union

PandasFeatureUnion(transformer_list=[('numerical_preprocessor',
                                      Pipeline(steps=[('numerical_selector',
                                                       FunctionTransformer(func=<function numerical_select at 0x000002279EF34AF0>)),
                                                      ('scaling',
                                                       StandardScalerDf())])),
                                     ('binary_preprocessor',
                                      Pipeline(steps=[('binary_selector',
                                                       FunctionTransformer(func=<function <lambda> at 0x00000227967BE310>)),
                                                      ('ohe',
                                                       OneHotEncodingDf())])),
                                     ('categorical_preprocessor',
                                      Pipeline(steps=[('categorical_selector',
                                                       FunctionTransformer(func=<function categorical_select at 0x0000022796FAEA60>)),
                                                      ('ohe',
                                                       OneHotEncodingDf())]))])

In [324]:
data

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Lying,Internet usage,Education,Gender,Left - right handed,Only child,Village - town,House - block of flats,random_values,random_binar
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,never,few hours a day,college/bachelor degree,female,right handed,no,village,block of flats,0.956590,True
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,sometimes,few hours a day,college/bachelor degree,female,right handed,no,city,block of flats,-0.237930,True
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,sometimes,few hours a day,secondary school,female,right handed,no,city,block of flats,0.121266,True
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,only to avoid hurting someone,most of the day,college/bachelor degree,female,right handed,yes,city,house/bungalow,0.323152,True
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,everytime it suits me,few hours a day,secondary school,female,right handed,no,village,house/bungalow,0.638346,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,5.0,2.0,5.0,2.0,2.0,5.0,4.0,4.0,4.0,3.0,...,everytime it suits me,few hours a day,secondary school,female,right handed,no,city,house/bungalow,-0.076750,True
1006,4.0,4.0,5.0,1.0,3.0,4.0,1.0,4.0,1.0,1.0,...,sometimes,less than an hour a day,masters degree,male,left handed,no,village,house/bungalow,2.183856,False
1007,4.0,3.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,1.0,...,only to avoid hurting someone,most of the day,secondary school,female,right handed,yes,city,block of flats,-0.394901,True
1008,5.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,4.0,1.0,...,sometimes,most of the day,college/bachelor degree,female,right handed,no,city,block of flats,-0.104756,True


In [325]:
fu_result = feature_union.fit_transform(data)

In [326]:
fu_result

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,x10_3.0,x10_4.0,x10_5.0,x10_nan,x11_1.0,x11_2.0,x11_3.0,x11_4.0,x11_5.0,x11_nan
0,0.403971,-0.393961,-0.951567,-1.131952,-0.114711,-0.763717,-1.398094,1.316567,1.045410,-0.992099,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.102691,0.805775,-0.951567,-1.131952,-1.044424,-1.562473,-0.604581,-0.406348,1.045410,1.193992,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.403971,2.005510,-0.951567,-0.253487,0.815003,0.833797,1.775957,-0.406348,1.045410,0.465295,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.403971,-0.393961,-0.951567,-1.131952,-1.044424,-1.562473,-1.398094,-1.267805,-1.487795,-0.992099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.403971,-0.393961,0.757855,0.624978,-0.114711,0.833797,0.188932,1.316567,-0.643394,-0.992099,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,0.403971,-1.593696,1.612566,-0.253487,-0.114711,1.632554,0.982445,0.455110,0.201008,0.465295,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1006,-1.102691,0.805775,1.612566,-1.131952,0.815003,0.833797,-1.398094,0.455110,-2.332197,-0.992099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1007,-1.102691,-0.393961,-1.806277,-1.131952,-0.114711,-0.763717,-0.604581,-0.406348,0.201008,-0.992099,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1008,0.403971,-0.393961,-0.096856,0.624978,-1.044424,0.035040,-1.398094,-0.406348,0.201008,-0.992099,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Попробуем теперь на test 

In [106]:
from sklearn.model_selection import train_test_split

data.dropna(inplace=True)

data_train, data_test = train_test_split(data, test_size=0.2, stratify=data[target])

feature_union.fit_transform(data_test).head(4)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,x10_1.0,x10_2.0,x10_3.0,x10_4.0,x10_5.0,x11_1.0,x11_2.0,x11_3.0,x11_4.0,x11_5.0
633,0.328345,0.807531,-0.133248,2.404459,0.859727,-1.542382,-1.356955,0.520787,-0.718278,-0.940885,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
25,0.328345,-0.454816,0.690465,-0.261354,-1.105363,-0.735597,0.174546,1.354046,-2.527275,-0.940885,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
334,0.328345,-1.717164,-0.956961,0.627250,0.859727,-0.735597,0.174546,-1.145731,-0.718278,-0.233922,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
281,0.328345,-0.454816,0.690465,-0.261354,0.859727,0.877971,-1.356955,-1.145731,0.186220,1.180005,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


# Using ColumnTransformer

In [107]:
# numerical_features

<div class="alert alert-info">
    
У `make_column_selector` есть пара удобных опций, чтобы отфильтровывать признаки вместо нас 

In [108]:
# data.dtypes

In [109]:
# make_column_selector(dtype_include="float64")(data)

## Выбираем все, кроме таргета

In [110]:
make_column_selector('^((?!Happiness).)*$', dtype_exclude=["object"])(data)

['Music',
 'Slow songs or fast songs',
 'Dance',
 'Folk',
 'Country',
 'Classical music',
 'Musical',
 'Pop',
 'Rock',
 'Metal or Hardrock',
 'Punk',
 'Hiphop, Rap',
 'Reggae, Ska',
 'Swing, Jazz',
 'Rock n roll',
 'Alternative',
 'Latino',
 'Techno, Trance',
 'Opera',
 'Movies',
 'Horror',
 'Thriller',
 'Comedy',
 'Romantic',
 'Sci-fi',
 'War',
 'Fantasy/Fairy tales',
 'Animated',
 'Documentary',
 'Western',
 'Action',
 'History',
 'Psychology',
 'Politics',
 'Mathematics',
 'Physics',
 'Internet',
 'PC',
 'Economy Management',
 'Biology',
 'Chemistry',
 'Reading',
 'Geography',
 'Foreign languages',
 'Medicine',
 'Law',
 'Cars',
 'Art exhibitions',
 'Religion',
 'Countryside, outdoors',
 'Dancing',
 'Musical instruments',
 'Writing',
 'Passive sport',
 'Active sport',
 'Gardening',
 'Celebrities',
 'Shopping',
 'Science and technology',
 'Theatre',
 'Fun with friends',
 'Adrenaline sports',
 'Pets',
 'Flying',
 'Storm',
 'Darkness',
 'Heights',
 'Spiders',
 'Snakes',
 'Rats',
 'Agein

In [111]:
data.shape

(676, 152)

# А теперь воспользуемся готовым

In [112]:
# preprocessor = ColumnTransformer(transformers=[
#     ('num', StandardScalerDf(), make_column_selector(r'^((?!Happiness).)*$', dtype_exclude=["object"])),
#     ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), make_column_selector(dtype_include="object"))
# ], remainder='passthrough')


# result = pd.DataFrame(preprocessor.fit_transform(data), columns=preprocessor.get_feature_names())
# result.head(5)

In [113]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScalerDf(), numerical_features),
    ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), to_dummies),
    #('ordinal', 'passthrough', categorical_features)
], remainder='drop', n_jobs=-1)


result = pd.DataFrame(preprocessor.fit_transform(data), columns=preprocessor.get_feature_names())
result.head(3)

,num__Music,num__Slow songs or fast songs,num__Dance,num__Folk,num__Country,num__Classical music,num__Musical,num__Pop,num__Rock,num__Metal or Hardrock,...,cat__x1_right handed,cat__x2_city,cat__x2_village,cat__x3_current smoker,cat__x3_former smoker,cat__x3_never smoked,cat__x3_tried smoking,cat__x4_drink a lot,cat__x4_never,cat__x4_social drinker
0,0.398786,-0.370599,-0.912166,-1.128520,-0.109944,-0.787370,-1.372560,1.332211,1.053076,-0.972567,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-1.286083,0.882026,-0.912166,-1.128520,-1.050734,-1.591392,-0.591541,-0.384804,1.053076,1.178323,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.398786,2.134651,-0.912166,-0.235219,0.830846,0.820673,1.751516,-0.384804,1.053076,0.461360,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [114]:
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('num', StandardScalerDf(),
                                 ['Music', 'Slow songs or fast songs', 'Dance',
                                  'Folk', 'Country', 'Classical music',
                                  'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
                                  'Punk', 'Hiphop, Rap', 'Reggae, Ska',
                                  'Swing, Jazz', 'Rock n roll', 'Alternative',
                                  'Latino', 'Techno, Trance', 'Opera', 'Movies',
                                  'Horror', 'Thriller', 'Comedy', 'Romantic',
                                  'Sci-fi', 'War', 'Fantasy/Fairy tales',
                                  'Animated', 'Documentary', 'Western', ...]),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['Gender', 'Left - right handed',
                                  'Village - town', 'Smoking', 'Alcohol'])])

In [115]:
# preprocessor.get_steps(

In [116]:
fu_result.columns

Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       ...
       'x10_3.0', 'x10_4.0', 'x10_5.0', 'x10_nan', 'x11_1.0', 'x11_2.0',
       'x11_3.0', 'x11_4.0', 'x11_5.0', 'x11_nan'],
      dtype='object', length=226)

In [117]:
preprocessor

ColumnTransformer(n_jobs=-1,
                  transformers=[('num', StandardScalerDf(),
                                 ['Music', 'Slow songs or fast songs', 'Dance',
                                  'Folk', 'Country', 'Classical music',
                                  'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
                                  'Punk', 'Hiphop, Rap', 'Reggae, Ska',
                                  'Swing, Jazz', 'Rock n roll', 'Alternative',
                                  'Latino', 'Techno, Trance', 'Opera', 'Movies',
                                  'Horror', 'Thriller', 'Comedy', 'Romantic',
                                  'Sci-fi', 'War', 'Fantasy/Fairy tales',
                                  'Animated', 'Documentary', 'Western', ...]),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['Gender', 'Left - right handed',
                                  'Village - town', 'Smoking', 'Alcohol'])])

# ML

In [118]:
#И другие, но вы про них уже знаете 

from sklearn.linear_model import LogisticRegression

In [119]:
pipeline = Pipeline([
    ("preprocessing", feature_union),
    #("preprocessing", preprocessor),
    ("modelling", LogisticRegression()) #Единственное правило это fit/predict
])

## Displaying pipelines

In [120]:
pipeline

Pipeline(steps=[('preprocessing',
                 PandasFeatureUnion(transformer_list=[('numerical_preprocessor',
                                                       Pipeline(steps=[('numerical_selector',
                                                                        FunctionTransformer(func=<function numerical_select at 0x0000022796FAEE50>)),
                                                                       ('scaling',
                                                                        StandardScalerDf())])),
                                                      ('binary_preprocessor',
                                                       Pipeline(steps=[('binary_selector',
                                                                        FunctionTransformer(func=<function <lambda> at 0x0000022794744280>)),
                                                                       ('ohe',
                                                                        OneHotEncodingDf())])),
                                                      ('categorical_preprocessor',
                                                       Pipeline(steps=[('categorical_selector',
                                                                        FunctionTransformer(func=<function categorical_select at 0x0000022796E50040>)),
                                                                       ('ohe',
                                                                        OneHotEncodingDf())]))])),
                ('modelling', LogisticRegression())])

In [121]:
pipeline.fit(data_train.drop(target, axis=1), data_train[target])

# pipeline.predict_proba(data_test)

KeyError: "['Happiness in life'] not in index"

# Важность признаков - feature Importance

In [126]:
# pipeline

In [127]:
feature_names = pipeline.named_steps[
    'preprocessing'].transform(data).columns

feature_names

Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       ...
       'x10_1.0', 'x10_2.0', 'x10_3.0', 'x10_4.0', 'x10_5.0', 'x11_1.0',
       'x11_2.0', 'x11_3.0', 'x11_4.0', 'x11_5.0'],
      dtype='object', length=208)

In [128]:
plt.figure(figsize=(15, 15))

featureImportance = pd.DataFrame({"feature": feature_names, 
                                  "importance": pipeline.named_steps['modelling'].coef_[2, : ]})

featureImportance.sort_values(["importance"], ascending=False, inplace=True)
sns.barplot(x=featureImportance["importance"], y=featureImportance['feature'])
plt.show()

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

<Figure size 1080x1080 with 0 Axes>

In [ ]:
pipeline.named_steps['preprocessing']

In [ ]:
pipeline.named_steps['preprocessing'].transformers[0][1].with_mean # Scaler param

<div class="alert alert-info">
    
* А что если есть текстовые признаки? `BagOfWords` / `TF-IDF`
* Временные ряды? `tsfresh` / `tsfel`
* Генарция новых признаков на лету - давайте реализуем


In [ ]:
data_path = "https://raw.githubusercontent.com/a-milenkin/mipt_data_science_2021/main/data/people_data_clustering.csv"
data = pd.read_csv(data_path, index_col=0)
print("Размеры датасета", data.shape)
data.head(6)

In [ ]:
sub_data = data[data['home_cost'] < 10.5**7]

sns.displot(sub_data,  x = 'happyness',  hue = 'profession', kind ='kde', height =10, aspect = 3);

<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой уже знакомую нам информацию про жителей некоторого города - Атлантас.

* `was_married` - Был ли когда-либо женат/замужем
* `children` - Количество детей
* `salary` - Уровень зарплаты
* `sex` - Пол (0 - женщина, 1 - мужчина)
* `age` - Возраст
* `home_cost` - Сумарная стоимость жилья/недвижимости
* `Professon` - Профессия
* `drive_info` - Есть ли водительские права
* `happyness` - уровень счастья по результатам опроса
    
</div>

In [ ]:
numerical_features = ['was_married', 'children', 'salary', 'sex', 'age', 'home_cost']
categorical_features = [] # ['profession']

target = 'happyness'


all_features = numerical_features + categorical_features

In [ ]:
#   Для категориальных - применяем SimpleImputer, а затем StandardScaler
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(score_func=f_classif, k="all")),
])

#   Для категориальных - применяем SimpleImputer, а затем OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    #("categorical", categorical_transformer, categorical_features),
])

# Создание конвейера препроцессора, который сначала преобразует данные
preprocessor = Pipeline(steps=[("data_transformer", data_transformer)])

classifier_pipline = Pipeline(
    steps=[("preprocessor", preprocessor), 
           ("classifier", LogisticRegression())])

classifier_pipline

In [129]:
classifier_pipline.fit(data[all_features], data[target])

ValueError: A given column is not a column of the dataframe

In [ ]:
data[all_features].info()

# Как сохранить всю эту махину?

In [ ]:
# И другие, но вы про них уже знаете
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Для сериализации моделей
from joblib import Parallel, delayed
from scipy import sparse
import pickle

# Save the pipeline

In [ ]:
with open("pipeline.pkl", "wb") as f:
    pickle.dump(search.best_estimator_, f)

# Load and predict

In [ ]:
with open("pipeline.pkl", 'rb') as f:
    pipeline_from_saved = pickle.load(f)

In [ ]:
pipeline_from_saved

In [ ]:
data_test.iloc[0]

In [ ]:
pipeline_from_saved.named_steps['preprocessing'].transform(data_test.iloc[:1])

In [ ]:
preds = pipeline_from_saved.predict(data_test)
print(classification_report(data_test[target], preds))

# Дополнительно

* Код/ноутбуки будут тут: https://github.com/a-milenkin/Otus_Open_Day

Пример хорошего ML проекта: https://github.com/a-milenkin/ml_project_example

**Но можно писать еще лучше!**

Задача: переписать модуль train_pipline используя Pipeline из Sklearn